# Assignment 1.1: Histogram Equalization

## Working with notebooks

The assignments for this course will be given as Jupyter notebooks like this one. They will contain programming assignments which are automatically graded.

An autograded assignment will typically ask you to complete the code of a Python function. It is important that you do not change the name, inputs or outputs of the function, because they will be tested automatically: stick closely to the assignment. The autograder converts your notebook to a Python file by concatenating all code in the notebook and testing the resulting Python file. It is therefore important that your code runs correctly and uninterrupted, so:

**Always use `Kernel > Restart & Run All` and check the results before handing in your assignment**.

Some general tips:

- Avoid using packages that are not already imported in the assignment template, and are not default Python packages.
- Avoid using "notebook magic" like `%matplotlib inline`; such commands do not work when the notebook is converted to a Python script.

## Working with Codegrade

The assignments for this course are automatically graded by a system called *Codegrade*. Our tests will run on your code automatically, and give grades based purely on the correctness of the output.

Some of the Codegrade tests will be run immediately, and you can view the output/results before the deadline (it may take a couple of minutes to show up). Use this to check that your code is working correctly, and that you are in the right direction for the assignments.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread as skimread

# Point Operators

In the lecture notes a point operator is defined as an operator:
$$ 
\forall \mathbf x\in \mathcal D: \quad g(\mathbf x) = \psi(f(\mathbf x)),
$$
where $\psi$ is a scalar function taking in a gray value (real number) and returning a gray value (real number).

We will consider three different implementations of a point operator:

1. writing a function based on an analytical expression (e.g. $\psi(v)=v^2$),
2. in case the range of an image $f$ is a discrete and finite set of values (e.g. all pixel values are encoded as 8 bit integers) we can use a **look-up table**, and
3. in case an analytical expression is not available but only an (approximate) graph of the function $\psi$ we can use table interpolation.

## Gamma Correction

To warm up for the histogram equalization implementation later on, we first consider **gamma correction**. Let $f$ be a scalar image with gray values in the range from zero to one, then the gamma corrected image $g$ is defined as:
$$ 
\forall \mathbf x\in \mathcal D: \quad g(\mathbf x) = (f(\mathbf x))^\gamma
$$
With the usual overloading of operators we write:
$$ 
g = f^\gamma
$$
First, you have to write a function for the gamma correction. (In this case writing a function is unnecessarily complex, but we need it for automatic grading.)

In [ ]:
def psi_gamma(f, gamma):
    """
    This function returns the array f^gamma: each value in the array f raised to the power gamma.

    Note: this function should not use (explicit) loops.
    """
    return f ** gamma

Assuming $f(\mathbf x)\in[0,1]\subset\mathbb{R}$ plot the graphs of $f^\gamma$ in one figure for values $\gamma \in\{ 0.3, 0.5, 0.7, 1, 1.43, 2.0, 3.33 \}$

In [ ]:
v = np.linspace(0, 1, 20)
f, _ = np.meshgrid(v, np.arange(4))
plt.subplot(2,1,1)
plt.imshow(f, vmin=0, vmax=1)
plt.title(r'$\gamma=1$')

plt.gray()
plt.axis('off')
plt.subplot(2,1,2)
gamma = 0.5
plt.imshow(psi_gamma(f, gamma), vmin=0, vmax=1);
plt.title('$\gamma=%5.2f$' % gamma)
plt.gray()
plt.axis('off');

Below an 'image' is shown where the gray values increase from zero to one going from left to right. Note that the perceptual difference between the lower gray values (darker tones) is smaller than the difference between the larger gray values.

In [ ]:
v = np.linspace(0, 1, 20)
f, _ = np.meshgrid(v, np.arange(4))
plt.subplot(2,1,1)
plt.imshow(f, vmin=0, vmax=1)
plt.title(r'$\gamma=1$')

plt.gray()
plt.axis('off')
plt.subplot(2,1,2)
gamma = 1.0
plt.imshow(psi_gamma(f, gamma), vmin=0, vmax=1); 
plt.title('$\gamma=%5.2f$' % gamma)
plt.gray()
plt.axis('off');

Looking at the $\gamma$-curves you should be able to guestimate a value of $\gamma$ that will result in a more perceptually uniform change of luminance from left to right. Use this value as the $\gamma$ value in the code above for the second row and observe the results.

The curve of 0.5 goes up faster in the beginning, making the difference between the darker values bigger, while decreasing the difference between larger gray values.

## Table Lookup

A (unary) pixel operator can be implemented as a table lookup in case the range of the image is finite and countable. For an 8 bit image this is evidently the case: the possible pixels values are integers in the range from 0 to 255.

For such an 8 bit image the gamma correction from the previous section will not work because the gamma correction method assumes that the range is from zero to one. Rewrite the `psi_gamma` function to take an extra keyword argument `max_val` for images in the range from 0 to `max_val`. Note that the resulting image should have the same range from zero to `max_val`. Without special care the resulting image will have floating point values; that is ok for now.

In [ ]:
def psi_gamma_scaled(v, gamma, max_val=1):
    v_scaled = v / max_val
    adjusted = v_scaled ** gamma
    return max_val * adjusted

Below we show two images. The original image 'trui' and a gamma corrected version of it. Note that we start with an image of dtype=uint8 and end up with an image of dtype=float64.

In [ ]:
# The first time you run these tests, the code downloads the standard computer vision example image "trui.png".
try:
    f = skimread('trui.png')
except FileNotFoundError:
    import requests
    image_file = requests.get("https://staff.fnwi.uva.nl/r.vandenboomgaard/ComputerVision/_images/trui.png")
    open("trui.png", "wb").write(image_file.content)
    f = skimread('trui.png')

plt.subplot(1,2,1)
plt.imshow(f, vmin=0, vmax=255)
plt.axis('off')
plt.title(r'dtype: %s [%d, %d]' % (f.dtype, f.min(), f.max()))
plt.subplot(1,2,2)
g = psi_gamma_scaled(f, 1.2, max_val=255)
plt.imshow(g, vmin=0, vmax=255)
plt.title(r'dtype: %s [%5.2f, %5.2f]' % (g.dtype, g.min(), g.max()))
plt.axis('off');

Of course we can simply cast the resulting image into a dtype=uint8 image again but that would not prevent that the calculations (and intermediate images) are dtype=float64. Using a look-up table is one way to prevent that floating point intermediate values are used. 

In [ ]:
def gamma_table_uint8(gamma):
    """returns a table (array) T of shape (256,)"""
    T = np.linspace(0, 1, 256, dtype=np.float64)
    T_scaled = T ** gamma + 1e-9
    T = 255 * T_scaled
    T = np.int_(T)
    return np.array(T, dtype=np.uint8)

The code below demonstrates the use of table lookup for gamma compression. Note that ``lut[f]`` does the table lookup for all pixels in the image. Read about the 'magic' of array indexing in Numpy to really understand this.

In [ ]:
plt.subplot(1,2,1)
plt.imshow(f, vmin=0, vmax=255)
plt.axis('off')
plt.title(r'dtype: %s [%d, %d]' % (f.dtype, f.min(), f.max()))
plt.subplot(1,2,2)
lut = gamma_table_uint8(1.2)
g = lut[f]
plt.imshow(g, vmin=0, vmax=255)
plt.title(r'dtype: %s [%d, %d]' % (g.dtype, g.min(), g.max()))
plt.axis('off');

## Interpolated Table Lookup

A lookup table can only be used in case the set of pixel (gray) values is finite and discrete. For a floating point image a lookup table is not feasible.

Consider a point operator characterized with the function $\psi$:

$$
g(\mathbf x) = \psi(f(\mathbf x))
$$

with $f(\mathbf x)\in[0,1]\subset\mathbb{R}$.

Now let $\psi$ not be given in an analytical form but with just a few samples $\psi(v_i)=\psi_i$ (for $i=1,\ldots,n$). We assume $v_1=0$ and $v_n=1$ and furthermore we assume the sequence $v_i$ is increasing.

An example is given in the Python code below:

In [ ]:
v = np.array([0, 0.25, 0.5, 0.75, 1])
psi = np.array([0, 0.1, 0.6, 0.9, 1])
plt.plot(v, psi, '-o');

Not only the five points $(v_i, \psi_i)$ are indicated but also the lines connecting the dots. This is called **linear interpolation**. In a subsequent lab exercise you will have to implement interpolation for 2D functions. Here we will use the Numpy ``interp`` function.

To get the value $\psi(0.6)$ through interpolation (given only the 5 points above) we can use:

In [ ]:
psi0p6 = np.interp(0.6, v, psi)
print(psi0p6)
plt.plot(v, psi, '-o')
axs = plt.axis()
plt.vlines(0.6, -1, psi0p6, linestyles='dotted')
plt.hlines(psi0p6, -1, 0.6, linestyles='dotted')
plt.axis(axs);

In [ ]:
f = plt.imread('trui.png')
print(f.dtype)
g = np.interp(f, v, psi)

plt.subplot(1,2,1)
plt.imshow(f); 
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(g); 
plt.axis('off');

Define the table $\psi_{\text{invert}}$ with as few points as possible to invert the image, i.e. $g = 1 - f$.

In [ ]:
def invert_table():
    """returns a tuple (v, psi) defining a table to be used for
    interpolated table lookup that inverts an image"""
    return [0, 1], [1, 0]

In [ ]:
v_inv, psi_inv = invert_table()
g = np.interp(f, v_inv, psi_inv)
plt.subplot(1,2,1)
plt.imshow(f); 
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(g); 
plt.axis('off');

Next you have to define the table for interpolated table lookup that thresholds an image:

$$
\psi_{t}(v) = \begin{cases}
0 &: v<t\\
1 &: v\geq t
\end{cases}
$$

In [ ]:
def threshold_table(t):
    v = [0, t, t, 1]
    psi = [0, 0, 1, 1]
    return np.array((v, psi))

In [ ]:
v_t, psi_t = threshold_table(0.5)
plt.subplot(1,2,1)
plt.imshow(f)
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(np.interp(f, v_t, psi_t))
plt.axis('off');

# Histogram Equalization

In the lecture notes the histogram equalization operation for an image with gray values in the range from zero to one is given as the point operator defined with the function:
$$
\psi(v) = H_f(v)
$$
where $H_f$ is the normalized cumulative histogram of the image $f$.

In [ ]:
def histogram_equalization(f, bins=100):
    h, be = np.histogram(f, range=(0,1), bins=bins)
    H = np.cumsum(h.astype(float)/sum(h))
    v = be
    psi = np.hstack(([0], H))
    return np.interp(f, v, psi)   

In [ ]:
bins = 64
f = plt.imread('trui.png')
plt.subplot(321)
plt.imshow(f)
plt.axis('off')

plt.subplot(322)
g = histogram_equalization(f, bins=bins)
plt.imshow(g)
plt.axis('off')

plt.subplot(323)
h_f, be_f = np.histogram(f, range=(0,1), bins=bins)
h_f = h_f/np.sum(h_f)
plt.bar(be_f[:-1], h_f, width=be_f[1]-be_f[0]);

plt.subplot(324)
h_g, be_g = np.histogram(g, range=(0,1), bins=bins)
h_g = h_g/np.sum(h_g)
plt.bar(be_g[:-1], h_g, width=be_g[1]-be_g[0])

plt.subplot(325)
H_f = np.cumsum(h_f)
plt.bar(be_f[:-1], H_f, width=be_f[1]-be_f[0])
plt.subplot(326)
H_g = np.cumsum(h_g)
plt.bar(be_g[:-1], H_g, width=be_g[1]-be_g[0]);
plt.subplots_adjust(bottom=0.01, top=1.2, right=1.5)

In order to do the histogram equilization on an 8 bit image (uint8 data type) you have to write the function ``histogram_equilization_table``. In that function you will need the histogram of the 8bit image. Instead of using the ``histogram`` function, it is better in this case to use the ``bincount`` function from numpy. Be sure to set the ``minlength`` parameter!


In [ ]:
def histogram_equalization_table(f):
    """return the lookup table psi_he for the histogram
    equalization of an image f (dtype=uint8). Note that the table
    should have shape (256,) and dtype=uint8"""
    f = np.ndarray.flatten(f)
    his = np.bincount(f, minlength=256)
    his = his.astype(float)/sum(his)
    his = np.array((255*np.cumsum(his)), dtype=np.uint8)
    return his

## Histogram Equalization in Practice

Make several (at least 4) pictures of the same scene/object from different points of view. Most likely the overall intensity distribution in the images is not equal due to the automatic lighting correction in your camera. Use histogram equalization to correct for this and comment on whether it does the intended job. In your notebook you have to show a 2 x n ‘matrix’ of images, in the top row the original images and in the bottom row the equalized images.

Your camera probably makes color images. Equalization as discussed works on gray value images (black and white images). You can use the function rgb2gray from sklearn.color to make black-and-white images out of your color images.

Put the images into the same directory as the notebook, and open them e.g. with `plt.imread`. When uploading your assignment, you can also submit the images: they will be placed in the same directory as your code when the code is run.


In [ ]:
from skimage.color import rgb2gray

for i in range(1, 5):
    test = plt.imread(f'{i}.jpg')
    gray = np.array(255*rgb2gray(test), dtype=np.uint8)
    equalization = histogram_equalization_table(gray)
    new_image = equalization[gray]
    plt.subplot(240+i)
    plt.imshow(gray)
    plt.axis('off')
    plt.subplot(244+i)
    plt.imshow(new_image)
    plt.axis('off')

The equalization does work, since there is much more detail in the third image. And the difference in contrast between the normal version of the first and equalized version is much larger, especially the black area is much darker. The bottom of the fourth image is also much darker, while the highlight is much lighter.

Bonus: a simple way to use equalization on color images (and have a color image as the result) is to first convert the color image to a colorspace that explicitly encodes the luminance (the gray value) and the color componenent. One example is the function rgb2hsv from sklearn.color. HSV stands for Hue, Saturation and Value, where value is the gray value component. Equalization of color images then amounts to: convert a rgb image into a hsv image, then equalize the v component and then convert the new hsv image back to rgb (with hsv2rgb).

In [ ]:
from skimage.color import rgb2hsv, hsv2rgb

def histogram_equalization_rgb(f):
    hsv = rgb2hsv(f)
    gray = np.array(255*hsv[:,:,2], dtype= np.uint8)
    eq = histogram_equalization_table(gray)
    equalized = eq[gray]
    hsv[:,:,2] = equalized.astype(float) / 255
    rgb = hsv2rgb(hsv)
    return rgb

# Assignment 1.2: Bilinear interpolation

In many image processing applications you have access to a number of image transformation methods. These include rotating, stretching and even shearing an image.

These methods can be used to create some artistic effect, but they are also useful in correcting things like camera rotation, perspective distortions, etc, that were introduced when the picture was taken.

The input for these methods is always a quadrilateral region of a discrete image $F$. That means that you only know the intensity values at discrete integral pixel positions.
The output consists of a rectangular image $G$ with axes aligned with the Cartesian coordinate axes $x$ and $y$.

In an arbitrary image transformation, there is no guarantee that an "input"-pixel will be positioned at a pixel in the "output" image as well. Rather, most of the time your output image pixels will "look at" image positions in the input image, which are "between" pixels.
<table><tr> 
    <td> <img src="attachment:pixels_orig.png"></td>
<td> <img src="attachment:pixels_rotated.png"></td>
<td> <img src="attachment:pixels_both.png"></td>
</tr></table>

So you need access to intensity values which are not on the sampling grid of the original image, *e.g.* the intensitiy value at position (6.4, 7.3).

In this Exercise you will examine and implement interpolation techniques which solve this problem.
In a later assignment you will implement image transformations employing these techniques, such as rotations and projective transformations. This requires determining the desired transformation from the image data or from user input.

The relevant theorie for this assignment can be found in the lecture notes in [section 1.4](https://staff.fnwi.uva.nl/r.vandenboomgaard/ComputerVision/LectureNotes/IP/Images/ImageInterpolation.html) and in the related theorie questions on ANS.

# Interpolation

The transformation algorithm needs a way to access the original image $F$ in
locations that are not on the sampling grid. We thus need a function pixelValue
that returns the value in the location $(x,y)$ even for non-integer coordinate
values.

### A note on indexing

There are multiple ways to interpret coordinates in an image. In mathematics, the x-axis usually goes right, and the y-axis goes up. If you have an image file `image`, typically `image[x]` selects a row, and `image[x][y]` then selects a column from that row. That means that the x-axis goes *down*, and the y-axis goes *right*.
Another way to do it is to interpret the pixel at coordinates $(x, y)$ as `image[y][x]`: then the x-axis goes right, and the y-axis goes down. (This is almost the same as the standard mathematical way of doing things, except the y-axis now goes down instead of up.) This is the convention we will be applying in this and future notebooks: in particular, the `pixel_value` function will now have inputs `F, y, x` rather than `F, x, y`, and in the case when `y, x` are integers in the domain of `F` it should now return `F[y][x]`.


## Exercises

### Exercise 1

Write a Python function `pixelValue` that returns the value of a pixel at real-valued coordinates $(x, y)$. Fill out the template given in the following cell.

You should implement nearest neighbour interpolation (`interpolation_method = 'nearest'`) and
bilinear interpolation (`interpolation_method = 'bilinear'`), as are described in the syllabus. Take the necessary steps to deal with the "border problem" (*i.e.* implement the function `inImage`). For now you do not yet have to implement any of the other `border_methods`. Instead for now we will simple use the `'constant'` `border_method` to return `cval` when we encounter a point outside of the image. 

In [ ]:
def in_image(image_shape, y, x):
    if not 0 <= y <= image_shape[0] - 1:
        return False
    if not 0 <= x <= image_shape[1] - 1:
        return False
    return True

def pixel_value(F, y, x, interpolation_method="nearest", border_method="constant", cval=0):
    if not in_image(F.shape, y, x):
        if border_method == "constant":
            return cval
        elif border_method == "closest":
            y_new = max(0, min(y, F.shape[0] - 1))
            x_new = max(0, min(x, F.shape[1] - 1))
            return pixel_value(
                F, y_new, x_new, interpolation_method, border_method, cval
            )
        elif border_method == "mirror":
            y_shape, x_shape = F.shape[0] - 1, F.shape[1] - 1
            new_y = np.abs((y - y_shape) % (2 * y_shape) - y_shape)
            new_x = np.abs((x - x_shape) % (2 * x_shape) - x_shape)
            return pixel_value(
                F, new_y, new_x, interpolation_method, border_method, cval
            )
        elif border_method == "wrap":
            new_y, new_x = y % F.shape[0], x % F.shape[1]
            if F.shape[0] - 1 < new_y < F.shape[0] or F.shape[1] - 1 < new_x < F.shape[1]:
                if interpolation_method == 'bilinear' or interpolation_method == 'linear':
                    a = new_x - int(new_x)
                    b = new_y - int(new_y)
                    y0 = int(new_y)
                    y1 = (y0 + 1) % F.shape[0]
                    x0 = int(new_x)
                    x1 = (x0 + 1) % F.shape[1]
                    return ((1 - a) * (1 - b) * pixel_value(F, y0, x0, 'nearest', border_method, cval)
                            + a * (1 - b) * pixel_value(F, y0, x1, 'nearest', border_method, cval)
                            + (1 - a) * b * pixel_value(F, y1, x0, 'nearest', border_method, cval)
                            + a * b * pixel_value(F, y1, x1, 'nearest', border_method, cval))
            return pixel_value(
                F,
                y % (F.shape[0]),
                x % (F.shape[1]),
                interpolation_method,
                border_method,
                cval,
            )

    if interpolation_method == "nearest":
        return F[int(y + 0.5), int(x + 0.5)]
    elif interpolation_method == "bilinear" or interpolation_method == 'linear':
        a = x - int(x)
        b = y - int(y)
        return ((1 - a) * (1 - b) * pixel_value(F, int(y), int(x), "nearest", border_method, cval)
                + a * (1 - b) * pixel_value(F, int(y), int(x) + 1, "nearest", border_method, cval)
                + (1 - a) * b * pixel_value(F, int(y) + 1, int(x), "nearest", border_method, cval)
                + a * b * pixel_value(F, int(y) + 1, int(x) + 1, "nearest", border_method, cval))

In [ ]:
from PIL import Image
import numpy as np

# The first time you run these tests, the code downloads the standard computer vision example image "trui.png".
try:
    im = Image.open("trui.png")
except FileNotFoundError:
    import requests
    image_file = requests.get("https://staff.fnwi.uva.nl/r.vandenboomgaard/ComputerVision/_images/trui.png")
    open("trui.png", "wb").write(image_file.content)
    im = Image.open("trui.png")
image = np.array(im)

assert image.shape == (256, 256), "Did you download the correct image?"
assert in_image(image.shape, 0, 0)
assert not in_image(image.shape, -0.5, -0.5)
assert in_image(image.shape, image.shape[0] - 1, image.shape[1] - 1)
assert not in_image(image.shape, image.shape[0] - 0.5, image.shape[1] - 0.5)
assert in_image(image.shape, 0.5, 0.5)
assert in_image(image.shape, 0, image.shape[1]/2)
assert in_image(image.shape, image.shape[0]/3, image.shape[1]/1.5)
assert not in_image(image.shape, image.shape[0] + 10, image.shape[1] - 5)

In [ ]:
from PIL import Image
import numpy as np
from math import isclose
image = np.array(Image.open("trui.png"))

sample_points = [(0, 0), (20, 20), (32.3, 39.6), (100.7, 21.2), (14.9, 142.9), 
                 (142.2, 98.1), (-2, 3), (1000.3, 20.5)]
nearest_results = [115, 143, 186, 176, 188, 149, 0, 0]

for (y, x), v in zip(sample_points, nearest_results):
    assert isclose(pixel_value(image, y, x, interpolation_method='nearest'), v), \
        f"For method nearest, and input point ({y},{x}), you give result " \
        f"{pixel_value(image, y, x, interpolation_method='nearest')} instead of {v}."



In [ ]:
from PIL import Image
import numpy as np
from math import isclose
image = np.array(Image.open("trui.png"))

sample_points = [(0, 0), (20, 20), (32.3, 39.6), (100.7, 21.2), (14.9, 142.9), 
                 (142.2, 98.1), (-2, 3), (1000.3, 20.5)]
bilinear_results = [115, 143, 184.24, 175.64, 187.7, 148.56, 0, 0]

for (y, x), v in zip(sample_points, bilinear_results):
    assert isclose(pixel_value(image, y, x, interpolation_method='bilinear'), v), \
        f"For method bilinear, and input point ({y},{x}), you give result " \
        f"{pixel_value(image, y, x, interpolation_method='bilinear')} instead of {v}."



### Exercise 2

Perform additional tests of the function `pixel_value` by using the `profile` function that samples an image at `n` equidistant points along a line from $(x_0, y_0)$ to $(x_1, y_1)$. Create a single plot that shows the profile along the same line, using both nearest-neighbor and bilinear interpolation, that clearly shows the difference between the two methods. (Hint: if you take `n` high enough, the difference should be clear.)

In [ ]:
def profile(image, y0, x0, y1, x1, n, method):
    xs = np.linspace(y0, y1, n)
    ys = np.linspace(x0, x1, n)
    return np.array([pixel_value(image, y, x, interpolation_method=method) for y, x in zip(ys, xs)])

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
image = np.array(Image.open("trui.png"))

nn = profile(image, 0, 0, 20, 0, 1000, 'nearest')
bl = profile(image, 0, 0, 20, 0, 1000, 'bilinear')
xs = np.linspace(0, 20, 1000)
plt.plot(xs, nn)
plt.plot(xs, bl);

### Exercise 3

Now return to the `pixel_value` function and implement the different `border_methods` for the border problem as described in [section 1.5](https://rvdboomgaard.github.io/ComputerVision_LectureNotes/LectureNotes/IP/Images/ImageExtrapolation.html) of the lecture notes.

Experiment with these new methods and describe your results.

In [ ]:
def profile_borders(image, y0, x0, y1, x1, n, border):
    xs = np.linspace(x0, x1, n)
    ys = np.linspace(y0, y1, n)
    return xs, np.array([pixel_value(image, y, x, 'bilinear', border) for y, x in zip(ys, xs)])

x0, x1 = 0, 512
y0, y1 = 0, 512
n = 10000
xs, closest = profile_borders(image, y0, x0, y1, x1, n, 'closest')
plt.plot(xs, closest, label='closest')
xs, mirror = profile_borders(image, y0, x0, y1, x1, n, 'mirror')
plt.plot(xs, mirror, label='mirror')
xs, wrap = profile_borders(image, y0, x0, y1, x1, n, 'wrap')
plt.plot(xs, wrap, label='wrap')
plt.legend();

As can be seen from the graph above, the difference between the different kinds of border methods is quite significant.
The closest function just constantly the last value that it encountered while going through the image. Wrapping repeats the same image infinitely, while mirroring flips the image.